# Convolution Neural Network - CNN Baseline

Convolution Neural Networks or CNN or Convnets are the current state of the art for most computer vision tasks.

This notebook will aply k-fold and use the concept of transfer learning.

Transfer learning:
- Train a CNN using 6 emotions dataset known as FER2013 dataset
- Strip out the top layers of that trained CNN architecture
- Freeze the weights of that architecture in hopes it learned high level features already
- Add back other layers that are trainable
- Train using our Smile dataset to learn specific features

In [1]:
# Import libraries and ignore warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
np.random.seed(2)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Plot architecture
import keras
import pydot
from keras.utils.vis_utils import model_to_dot

from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing import image

import os
from PIL import Image

Using TensorFlow backend.


In [ ]:
def build_model_1():
    """
    Three convolutions no dropout
    """

    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3))) # Change it to 128, 128
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))

    # Feed to a densily connected layer for prediction
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',
          optimizer=optimizers.Adam(lr=0.001),
          metrics=['acc'])
    
    return model

# Data  
  
The first step is to import images and store their **pixel** values in a dataframe. This will allow us to create 10 fold by using the index of the rows.  
  
We will also **shuffle** the dataframe because the first 200 images are smilling and next 200 images are neutral.

In [4]:
# Read the annotations file that contains the label and the image file name
labels = pd.read_csv('./SMILE_Dataset/annotations.csv', header=None, names=['fname','label'])

# Shuffle data
labels = labels.sample(frac=1).reset_index()

# Use a list comprehension to loop over image file names and import one by one and store pixel values
x = np.array([image.img_to_array(image.load_img('./SMILE_Dataset/all/'+fname, target_size=(64, 64))) for fname in labels['fname']])

# Because the names are strings, the neural network only takes in numerical formats so we will one-hot encode the label
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels['label'])
y = integer_encoded

Now we have two variables.  
  
x: all of the values for our images  
y: all of the labels (0:1)
  
Now we have finished working with the data. Let's define an architecture for our CNN.  

# K-Fold

In [10]:
# All classification reports will be added here. When we are done we can average the f1 scores
reports = []

# Apply stratified K-fold ith 10 splits. Stratified means the same distribution of classes than the whole dataset
# In this case, 50-50
kf = StratifiedKFold(n_splits=10)

# Just for printing purposes
id = 1

for train_index, test_index in kf.split(x,y):
    print('Kfold iteration {}/10'.format(id))
    print('Total images: {} ---- Train images: {} ---- Test images: {}'.format(len(x),len(train_index),len(test_index)))

    id += 1 
    
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
        
    model = build_model_1()
    
    datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=10, # randomly rotate images in the range (degrees, 0 to 180)
                                 width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
                                 height_shift_range=0.1, 
                                 shear_range=0.1,
                                 zoom_range=0.1)   
    
    datagen.fit(X_train)

    # Secret sauce to get 3-5 % accuracy more
    # Adjust the learning rate over time. (Like we saw in class!)
    # The learning rate determines the size of the steps taken during the gradient descent process.
    
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
    
    # Used to prevent overfitting. 
    # es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
    
    history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = 20), epochs = 20, 
                              validation_data = (X_test,y_test), steps_per_epoch=len(X_train) / 20,
                              callbacks=[learning_rate_reduction])

    
    y_pred = model.predict(X_test)
    y_pred = [np.round(p[0]) for p in y_pred]
    
    print(classification_report(y_test, y_pred))
    reports.append(classification_report(y_test, y_pred,output_dict=True))
    
#     if id == 3:
#         break

Kfold iteration 1/10
Total images: 400 ---- Train images: 360 ---- Test images: 40
Epoch 1/20
18/18 [==============================] - 4s 219ms/step - loss: 0.6951 - acc: 0.5000 - val_loss: 10.1461 - val_acc: 0.5000
Epoch 2/20
18/18 [==============================] - 3s 187ms/step - loss: 0.6941 - acc: 0.5000 - val_loss: 8.5083 - val_acc: 0.5000
Epoch 3/20
18/18 [==============================] - 3s 189ms/step - loss: 0.6931 - acc: 0.5000 - val_loss: 6.7178 - val_acc: 0.5000
Epoch 4/20
18/18 [==============================] - 3s 193ms/step - loss: 0.6925 - acc: 0.5028 - val_loss: 4.8669 - val_acc: 0.5000
Epoch 5/20
18/18 [==============================] - 3s 190ms/step - loss: 0.6912 - acc: 0.5000 - val_loss: 3.1809 - val_acc: 0.5500
Epoch 6/20
18/18 [==============================] - 3s 193ms/step - loss: 0.6904 - acc: 0.5389 - val_loss: 1.7923 - val_acc: 0.6500
Epoch 7/20
18/18 [==============================] - 3s 194ms/step - loss: 0.6884 - acc: 0.5972 - val_loss: 1.0869 - val_acc:

Epoch 16/20
18/18 [==============================] - 4s 195ms/step - loss: 0.6779 - acc: 0.8056 - val_loss: 1.2253 - val_acc: 0.8250
Epoch 17/20
18/18 [==============================] - 3s 193ms/step - loss: 0.6757 - acc: 0.8000 - val_loss: 1.3205 - val_acc: 0.8500
Epoch 18/20
18/18 [==============================] - 4s 198ms/step - loss: 0.6748 - acc: 0.8222 - val_loss: 1.3213 - val_acc: 0.8250
Epoch 19/20
18/18 [==============================] - 4s 201ms/step - loss: 0.6730 - acc: 0.8417 - val_loss: 1.3867 - val_acc: 0.8250
Epoch 20/20
18/18 [==============================] - 4s 204ms/step - loss: 0.6744 - acc: 0.8333 - val_loss: 1.4049 - val_acc: 0.8250
              precision    recall  f1-score   support

           0       0.81      0.85      0.83        20
           1       0.84      0.80      0.82        20

    accuracy                           0.82        40
   macro avg       0.83      0.82      0.82        40
weighted avg       0.83      0.82      0.82        40

Kfold it

18/18 [==============================] - 4s 207ms/step - loss: 0.6869 - acc: 0.6889 - val_loss: 0.5604 - val_acc: 0.8000
Epoch 9/20
18/18 [==============================] - 4s 226ms/step - loss: 0.6862 - acc: 0.6972 - val_loss: 0.5370 - val_acc: 0.8250
Epoch 10/20
18/18 [==============================] - 4s 207ms/step - loss: 0.6844 - acc: 0.7250 - val_loss: 0.4774 - val_acc: 0.8500
Epoch 11/20
18/18 [==============================] - 4s 205ms/step - loss: 0.6856 - acc: 0.7222 - val_loss: 0.4730 - val_acc: 0.8750
Epoch 12/20
18/18 [==============================] - 4s 209ms/step - loss: 0.6848 - acc: 0.7500 - val_loss: 0.4897 - val_acc: 0.8500
Epoch 13/20
18/18 [==============================] - 4s 207ms/step - loss: 0.6830 - acc: 0.7306 - val_loss: 0.4940 - val_acc: 0.8500
Epoch 14/20
18/18 [==============================] - 4s 198ms/step - loss: 0.6829 - acc: 0.7861 - val_loss: 0.5280 - val_acc: 0.8500
Epoch 15/20
18/18 [==============================] - 4s 205ms/step - loss: 0.6827 

Epoch 1/20
18/18 [==============================] - 5s 257ms/step - loss: 0.6936 - acc: 0.5167 - val_loss: 4.8867 - val_acc: 0.4500
Epoch 2/20
18/18 [==============================] - 4s 221ms/step - loss: 0.6897 - acc: 0.5417 - val_loss: 4.9228 - val_acc: 0.5000
Epoch 3/20
18/18 [==============================] - 4s 215ms/step - loss: 0.6917 - acc: 0.5333 - val_loss: 4.1728 - val_acc: 0.5250
Epoch 4/20
18/18 [==============================] - 4s 211ms/step - loss: 0.6886 - acc: 0.5722 - val_loss: 3.9729 - val_acc: 0.5500
Epoch 5/20
18/18 [==============================] - 4s 218ms/step - loss: 0.6877 - acc: 0.5889 - val_loss: 3.2911 - val_acc: 0.5750
Epoch 6/20
18/18 [==============================] - 4s 220ms/step - loss: 0.6867 - acc: 0.5778 - val_loss: 2.8303 - val_acc: 0.6000
Epoch 7/20
18/18 [==============================] - 4s 210ms/step - loss: 0.6873 - acc: 0.6083 - val_loss: 2.6076 - val_acc: 0.6250
Epoch 8/20
18/18 [==============================] - 4s 236ms/step - loss: 0.

In [11]:
# We loop over all reports (1 per fold) and then compute the average of all weighted f1 scores
final_f1_score = np.mean([rep['weighted avg']['f1-score'] for rep in reports])

print('Final F1-Score is: {}%'.format(np.round(final_f1_score*100,2)))

Final F1-Score is: 84.07%
